# Metodos Numericos 1

## Importando pacotes

In [26]:
import numpy as np
from methods.bissection import Bissection
from methods.false_position import FalsePosition
from methods.fixed_point import FixedPoint
from methods.newton_raphson import NewtonRaphson
from methods.secant import Secant

## Hiperparametros gerais

Serão utilizados em toda parte do notebook

In [27]:
funcao:str = "np.exp(-x) - x"
xU:np.float64 = np.float64(0.0)
xL:np.float64 = np.float64(1.0)
tol:np.float64 = np.float64(0.0005)
max_iter:int = 5

## Modelos Numericos

### Metodos Intervalares

#### Bisseccao

In [28]:
bissection = Bissection(xL, xU, tol, max_iter)
bissection.execute_method(f)

np.float64(0.59375)

#### Falsa Posicao

In [29]:
falsePosition = FalsePosition(xL, xU, tol, max_iter)
falsePosition.execute_method(funcao)

np.float64(0.567150214240495)

### Metodos Abertos

#### Ponto Fixo

In [30]:
# f(x) = x - 5
# f'(x) = 1
# estrutura de g(x)
# ==> g(x) = x - (f(x) / f'(x))
g_from_function:str = "x - (x - 5)"

fixedPoint = FixedPoint(xL, tol, max_iter)
fixedPoint.execute_method(g_from_function)

np.float64(5.0)

#### Newton Raphson

In [31]:
# f(x) = x - 5
# f'(x) = 1
derivative:str = "-exp(-x) - 1"

newtonRaphson = NewtonRaphson(xL, tol, max_iter)
newtonRaphson.execute_method(funcao, derivative)

np.float64(0.5671432904097838)

#### Secante

In [32]:
secant = Secant(xL, xU, tol, max_iter)
secant.execute_method(funcao)

np.float64(0.5671432904100637)